# Using Weights & Biases Weave with AWS Bedrock

In this notebook you will learn to use our newly realeased tool for the LLM practitioner

You can use [Weave](https://wandb.github.io/weave/) to:

- Log and debug language model inputs, outputs, and traces
- Build rigorous, apples-to-apples evaluations for language model use cases
- Organize all the information generated across the LLM workflow, from experimentation to evaluations to production

## Setup

In [1]:
!pip install -Uq wandb weave


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [1]:
import json
import boto3
from botocore.exceptions import ClientError

## log to your W&B account

In [2]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: capecape. Use `wandb login --relogin` to force relogin


True

## create a W&B project to store your results

In [6]:
import weave
weave.init('bedrock-weave')

Logged in as W&B user capecape.
View Weave data at https://wandb.ai/capecape/bedrock-weave/weave


Decorate your function call, that's it!

In [7]:
@weave.op() # <- just add this 😎
def generate_text(
    model_id: str, 
    prompt: str, 
    max_tokens: int=400,
    temperature: float=0.7,
    top_p: float=0.7,
    top_k: int=50,
) -> dict:
    bedrock = boto3.client(service_name='bedrock-runtime')


    body = json.dumps({
        "prompt": prompt,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "top_p": top_p,
        "top_k": top_k,
    })
    
    response = bedrock.invoke_model(
        body=body,
        modelId=model_id
    )

    response_body = json.loads(response.get('body').read())

    outputs = response_body.get('outputs')
    
    return outputs

Let's first try using Mistral 7B

In [8]:
model_id = 'mistral.mistral-7b-instruct-v0:2'

prompt = """<s>[INST] Create a Vegan Carbonara Recipe[/INST] """

outputs = generate_text(model_id, prompt)

for index, output in enumerate(outputs):

    print(f"Output {index + 1}\n----------")
    print(f"Text:\n{output['text']}\n")
    print(f"Stop reason: {output['stop_reason']}\n")

🍩 https://wandb.ai/capecape/bedrock-weave/r/call/4dff4369-4d76-45f4-a0d5-a4c80298677d
Output 1
----------
Text:
 Title: Vegan Carbonara with Creamy Cashew Sauce

Prep Time: 15 minutes
Cook Time: 10 minutes
Total Time: 25 minutes
Servings: 4

Ingredients:
- 1 pound (450g) spaghetti or your preferred pasta
- 1 1/2 cups (375ml) raw, unsalted cashews, soaked in water for at least 2 hours and drained
- 1 1/2 cups (375ml) vegetable broth
- 1/2 cup (120ml) unsweetened almond milk
- 1/2 cup (120g) nutritional yeast
- 2 tbsp (30ml) olive oil, divided
- 1 large onion, finely chopped
- 4 cloves garlic, minced
- 1 cup (240g) cooked chickpeas or firm tofu, drained and rinsed
- 1/2 cup (120g) steamed or roasted broccoli
- 1/2 cup (120g) sliced mushrooms
- 1/2 cup (120g) sliced zucchini
- 1/2 cup (120g) chopped kale
- Salt and freshly ground black pepper, to taste
- 1 tbsp (15ml) lemon juice
- 1/2 cup (120g) vegan bacon or pancetta (optional)
- 1/4 cup (60g) chopped fresh parsley

Instructions:

1. C

## Translation project

Let's use these powerful LLMs to translate the documentation of Weights & Biases

In [23]:
from pathlib import Path
from dataclasses import dataclass

system_prompt = """
# Instructions

You are a documentation translation assistant from English to {output_language}. We are translating valid docusaurus flavored markdown. Some rules to remember:

- Do not add extra blank lines.
- The results must be valid docusaurus markdown
- It is important to maintain the accuracy of the contents but we don't want the output to read like it's been translated. So instead of translating word by word, prioritize naturalness and ease of communication.
- In code blocks, just translate the comments and leave the code as is.


## Formatting Rules

Do not translate target markdown links. Never translate the part of the link inside (). For instance here [https://wandb.ai/site](https://wandb.ai/site) do not translate anything, but on this, you should translate the [] part:
[track metrics](./guides/track), [create logs](./guides/artifacts).
"""
human_prompt = """
Here is a chunk of documentation in docusaurus Markdown format to translate. Return the translation only, without adding anything else. 
<Markdown start>
{md_chunk}
<End of Markdown>
"""

@dataclass
class PromptTemplate:
    system_prompt: str
    human_prompt: str
    language: str
    
    def format_mistral_instruct(self, md_chunk):
        "A formatting function for Mistral Instruct models"
        system_prompt = self.system_prompt.format(output_language=self.language)
        human_prompt = self.human_prompt.format(md_chunk=md_chunk)
        prompt = f"<s>[INST] {system_prompt}\n{human_prompt}[/INST] """
        return prompt

In [24]:
prompt_template = PromptTemplate(system_prompt, human_prompt, "spanish")

let's read 2 files from our documentation

In [25]:
doc = Path("./docs/quickstart.md").read_text()

We can now call the model with this new prompt:

In [26]:
output = generate_text(model_id, prompt_template.format_mistral_instruct(doc), max_tokens=2048)
print(f"> Text:\n{output[0]['text'][0:200]}\n")  # print first 200 chars
print(f"> Stop reason: {output[0]['stop_reason']}\n")

🍩 https://wandb.ai/capecape/bedrock-weave/r/call/e87a266d-2ddf-41ea-b38b-5ec8c310c240
> Text:
---
description: Inicio rápido de W&B.
displayed_sidebar: por defecto
---

import Tabs from '@theme/Tabs';
import TabItem from '@theme/TabItem';

# Inicio rápido

Instale W&B y monitoree sus experimen

> Stop reason: length



We can improve our weave experience by creating a `Model`

In [27]:
from weave import Model

class MistralInstruct(Model):
    prompt_template: PromptTemplate
    temperature: float=0.7
    max_tokens: int=2048
    model_id: str='mistral.mistral-7b-instruct-v0:2'

    @weave.op()
    def predict(self, document: str) -> dict:
        output = generate_text(
             model_id, 
             self.prompt_template.format_mistral_instruct(document), 
             temperature=self.temperature,
             max_tokens=self.max_tokens,
        )
        return output[0]['text']

In [28]:
model = MistralInstruct(prompt_template=prompt_template)

In [29]:
out = model.predict(doc)

🍩 https://wandb.ai/capecape/bedrock-weave/r/call/ed0d66aa-d021-4667-b66a-ca29cc99afdd


In [30]:
print(out[0:400])

---
description: Rápido inicio con W&B.
displayed_sidebar: default
---

import Tabs from '@theme/Tabs';
import TabItem from '@theme/TabItem';

# Rápido inicio

Instale W&B y monitoree tus experimentos de aprendizaje automático en minutos.

## 1. Crea una cuenta y instala W&B
Antes de empezar, asegúrate de crear una cuenta y de instalar W&B:

1. [Regístrate](https://wandb.ai/site) gratis en [https:
